In [ ]:
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:

import os

filepath = "/content/drive/MyDrive/alphafold_results/"
files = os.listdir("/content/drive/MyDrive/alphafold_results")

import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/alphafold_results/train.csv")


In [ ]:
## Extract and copy pdb files

for i in range(len(files)):
    !unzip -o {filepath + files[i]} -d {files[i].split(".")[0]}

import shutil
for i in os.listdir():
    for j in os.listdir(i):
            
        if j[-4:] == ".pdb":
            print(i+ "/" + j)
            shutil.copy(i+ "/" + j, "/content/drive/MyDrive/alphafold_results")
            shutil.copy(i+ "/" + j[:10] + ".a3m", "/content/drive/MyDrive/alphafold_results")
        


test_b1165/test_b1165_unrelaxed_rank_1_model_1.pdb
test_95853/test_95853_unrelaxed_rank_1_model_1.pdb
test_5cf89/test_5cf89_unrelaxed_rank_1_model_1.pdb
test_ee1db/test_ee1db_unrelaxed_rank_1_model_1.pdb
test_1e44e/test_1e44e_unrelaxed_rank_1_model_1.pdb
test_9b167/test_9b167_unrelaxed_rank_1_model_1.pdb
test_a0fc1/test_a0fc1_unrelaxed_rank_1_model_1.pdb
test_b8980/test_b8980_unrelaxed_rank_1_model_1.pdb
test_3d223/test_3d223_unrelaxed_rank_1_model_1.pdb
test_c91b3/test_c91b3_unrelaxed_rank_1_model_1.pdb
test_d456b/test_d456b_unrelaxed_rank_1_model_1.pdb
test_ae7c8/test_ae7c8_unrelaxed_rank_1_model_1.pdb
test_3fa84/test_3fa84_unrelaxed_rank_1_model_1.pdb
test_ee447/test_ee447_unrelaxed_rank_1_model_1.pdb
test_65954/test_65954_unrelaxed_rank_1_model_1.pdb
test_f307d/test_f307d_unrelaxed_rank_1_model_1.pdb
test_3e9e4/test_3e9e4_unrelaxed_rank_1_model_1.pdb
test_e0709/test_e0709_unrelaxed_rank_1_model_1.pdb
test_a36f8/test_a36f8_unrelaxed_rank_1_model_1.pdb
test_c1661/test_c1661_unrelaxed

In [ ]:
import numpy as np
def preprocess_to_feature_input(filename, label):
    with open (filename, "r") as myfile:
        data = myfile.read().splitlines()

    unique_atoms = np.unique([i.split()[0] for i in data] + [i.split()[1] for i in data])
    atoms2id ={i:ind for ind, i in enumerate(unique_atoms)}
    atoms2id
    id2atoms = {ind:i for ind, i in enumerate(unique_atoms)}
    deg_temp = np.unique([i.split()[0] for i in data] + [i.split()[1] for i in data], return_counts = True)
    atom_degree = {atoms2id[deg_temp[0][i]]:deg_temp[1][i] for i in range(len(deg_temp[0]))}
    edges = [[atoms2id[i.split()[0]], atoms2id[i.split()[1]]] for i in data]
    bond_length = {str(atoms2id[i.split()[0]])+"_" + str(atoms2id[i.split()[1]]):float(i.split()[5][:-1]) for i in data}
    id2atomName = {atoms2id[i]: i.split(":")[-1] for i in atoms2id}


    res = {"edges": edges, "features_degree": atom_degree, "bond_length" : bond_length, "atomName":id2atomName, "id2atoms":id2atoms, "target":label}
    np.save(f"{filename[:-4]}_out.npy", res)

    return res
  


In [ ]:
!pip install graphein

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 185 kB 5.6 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 58 kB 4.9 MB/s 
     |████████████████████████████████| 11.2 MB 51.7 MB/s 
     |████████████████████████████████| 237 kB 60.9 MB/s 
     |████████████████████████████████| 2.6 MB 48.5 MB/s 
     |████████████████████████████████| 197 kB 48.2 MB/s 
     |████████████████████████████████| 878 kB 51.8 MB/s 
     |████████████████████████████████| 636 kB 49.4 MB/s 
     |████████████████████████████████| 72 kB 1.0 MB/s 
     |████████████████████████████████| 47 kB 3.9 MB/s 
     |████████████████████████████████| 48 kB 4.7 MB/s 
     |████████████████████████████████| 144 kB 46.1 MB/s 
     |████████████████████████████████| 965 kB 49.7 MB/s 
     |███████████████████████

In [ ]:
## Extract the features of graph from pdb files using graphein and save into numpy dictionary

from graphein.protein.graphs import construct_graph
from graphein.protein.edges.atomic import add_atomic_edges
from graphein.protein.config import ProteinGraphConfig
import networkx as nx

params_to_change = {"granularity": "atom", "edge_construction_functions": [add_atomic_edges]}

config = ProteinGraphConfig(**params_to_change)
config.dict()


files = os.listdir("/content/drive/MyDrive/alphafold_results")
for f in files:
    if f[-4:] == ".pdb":

        g = construct_graph(config=config, pdb_path="/content/drive/MyDrive/alphafold_results/" + str(f))
        txt_filename = f[:-4] + ".txt"
        print(txt_filename)
        nx.write_edgelist(g, txt_filename)
        shutil.copy(f[:-4] + ".txt", "/content/drive/MyDrive/alphafold_results/")

        f = open("/content/drive/MyDrive/alphafold_results/" + str(f[:10]+".a3m"))
        fnd = f.readlines()[2]
        lbl = df["tm"][df["protein_sequence"]+"\n"==fnd].values[0]
        
        gh = preprocess_to_feature_input("/content/drive/MyDrive/alphafold_results/"+txt_filename, lbl)


Output()

test_a0fc1_unrelaxed_rank_1_model_1.txt


Output()

test_08cf8_unrelaxed_rank_1_model_1.txt


Output()

...

test_45d42_unrelaxed_rank_1_model_1.txt


Output()

test_fa851_unrelaxed_rank_1_model_1.txt


Output()

test_e0709_unrelaxed_rank_1_model_1.txt


In [ ]:
import shutil
shutil.copy(filepath+files[0], "/content")

'/content/test_813b2.result.zip'

In [ ]:
seq = []
for i in os.listdir('/content'):
    if(i[:4]=="test"):
        f = open("/content/" +i + "/" + i + ".a3m", 'r')
        seq.append((f.readlines()[2]))

In [ ]:
seq[0]

'AINQMTKTTIAEQASLQQKALFDQVLPAERYNNALAQSCYLVTAPELGKGEHRVYIAKQDDKPVAAVLEATAPDGYSGAIQLLVGADFNGTVLGTRVTEHHETPGLGDKIELRLSDWITHFAGKKISGADDAHWAVKKDGGDFDQFTGATITPRAVVNAVKRAGLYAQTLPAQLSQLPACG\n'

In [ ]:
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/alphafold_results/train.csv")
df

,seq_id,protein_sequence,pH,data_source,tm
0,0,AAAAKAAALALLGEAPEVVDIWLPAGWRQPFRVFRLERKGDGVLVG...,7.0,doi.org/10.1038/s41592-020-0801-4,75.7
1,1,AAADGEPLHNEEERAGAGQVGRSLPQESEEQRTGSRPRRRRDLGSR...,7.0,doi.org/10.1038/s41592-020-0801-4,50.5
2,2,AAAFSTPRATSYRILSSAGSGSTRADAPQVRRLHTTRDLLAKDYYA...,7.0,doi.org/10.1038/s41592-020-0801-4,40.5
3,3,AAASGLRTAIPAQPLRHLLQPAPRPCLRPFGLLSVRAGSARRSGLL...,7.0,doi.org/10.1038/s41592-020-0801-4,47.2
4,4,AAATKSGPRRQSQGASVRTFTPFYFLVEPVDTLSVRGSSVILNCSA...,7.0,doi.org/10.1038/s41592-020-0801-4,49.5
...,...,...,...,...,...
31385,31385,YYMYSGGGSALAAGGGGAGRKGDWNDIDSIKKKDLHHSRGDEKAQG...,7.0,doi.org/10.1038/s41592-020-0801-4,51.8
31386,31386,YYNDQHRLSSYSVETAMFLSWERAIVKPGAMFKKAVIGFNCNVDLI...,7.0,doi.org/10.1038/s41592-020-0801-4,37.2
31387,31387,YYQRTLGAELLYKISFGEMPKSAQDSAENCPSGMQFPDTAIAHANV...,7.0,doi.org/10.1038/s41592-020-0801-4,64.6
31388,31388,YYSFSDNITTVFLSRQAIDDDHSLSLGTISDVVESENGVVAADDAR...,7.0,doi.org/10.1038/s41592-020-0801-4,50.7


In [ ]:
df["tm"][df["protein_sequence"]+"\n"==seq[4]].values[0]

41.3

In [ ]:
df["protein_sequence"][0]

'AAAAKAAALALLGEAPEVVDIWLPAGWRQPFRVFRLERKGDGVLVGMIKDAGDDPDVTHGAEIQAFVRFASEDRLEGGEGVGVVTKPGLGVPVGEPAINPVPRRMIWEAVREVTERPLAVTIAIPGGEELAKKTLNPRLGILGGLSVLGTTGVVKPYSTSAFRMSVVQAVGVARANGLLEIAATTGGKSERFAQRLLPHLPEMAFIEMGDFVGDVLRAARKVGVEVVRVVGMIGKISKMADGKTMTHAAGGEVNLSLLLSLLKEAGASPKALKEAEGAATARRFLEIALEEGLELFFVNLVRLAQEKLQAYIGERPFVSVALTDFDEGRCLAAWPDREVYR'

In [ ]:
seq[0]

'AINQMTKTTIAEQASLQQKALFDQVLPAERYNNALAQSCYLVTAPELGKGEHRVYIAKQDDKPVAAVLEATAPDGYSGAIQLLVGADFNGTVLGTRVTEHHETPGLGDKIELRLSDWITHFAGKKISGADDAHWAVKKDGGDFDQFTGATITPRAVVNAVKRAGLYAQTLPAQLSQLPACG\n'